# Подгонка ARIMA (пакет [`statsmodels`](https://www.statsmodels.org/stable/index.html))

Для подгонки используем классы 
* [ARIMA](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMA.html#statsmodels.tsa.arima.model.ARIMA) (специфицированная модель)
* [ARIMAResult](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMAResults.html#statsmodels.tsa.arima.model.ARIMAResults) (подогнанная модель)

In [ ]:
import numpy as np
import pandas as pd

import statsmodels.tsa.api as tsa
from statsmodels.stats.diagnostic import het_arch, acorr_ljungbox

import pandas_datareader.data as web

# настройки визуализация
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

## Подгонка ARIMA заданного порядка без сноса

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) недельные данные по Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity (Symbol [`WGS10YR`](https://fred.stlouisfed.org/series/WGS10YR)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `y`



In [ ]:
y = web.DataReader(name='WGS10YR', data_source='fred', start='2000-01-01', end='2023-12-31')

Подгоним модель ARIMA(2,1,1) без сноса для `y`

Спецификация

$$
	(1-\phi_1L-\phi_2 L^2)(1-L) y_t=u_t+\theta u_{t-1}
$$

In [ ]:
# спецификация модели
arima_mod = tsa.ARIMA(y, order=(2,1,1), trend='n', missing='drop')
# подгонка модели на данных
arima_res = arima_mod.fit()
# выводим результаты подгонки
arima_res.summary(alpha=0.05)

## Диагностика подогнанной модели

Базовые графики

In [ ]:
arima_res.plot_diagnostics(lags=15)

plt.show()

Тест на серийную корреляцию (Ljung-Box test). Выберем число лагов 7

In [ ]:
acorr_ljungbox(arima_res.resid[1:] , lags=[7], model_df=2+1)

Тест на гетероскедастичность (на ARCH-эффекты)

In [ ]:
lm_stat, lm_pval, f_stat, f_pval = het_arch(arima_res.resid[1:], nlags=7, ddof=2+1)

lm_stat, lm_pval

## Подгонка ARIMA заданного порядка со сносом

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) квартальные данные по ВВП (Symbol [`GDP`](https://fred.stlouisfed.org/series/GDP)) с 1990 Q1 по 2023 Q4 и создадим датафрейм `y=log(GDP)`

In [ ]:
y = np.log(web.DataReader(name='GDP', data_source='fred', start='1990-1', end='2023-12' ) )

Подгоним модель ARIMA(1,1,1) со сносом для `y` 

Спецификация

$$
	(1-\phi L)(1-L) y_t=\alpha_0+u_t+\theta u_{t-1}
$$

In [ ]:
# спецификация модели
arima_drift_mod = tsa.ARIMA(y, order=(1,1,1), trend='t', missing='drop')
# подгонка модели на данных
arima_drift_res = arima_drift_mod.fit()
# выводим результаты подгонки
arima_drift_res.summary(alpha=0.05)

In [ ]:
arima_drift_res.plot_diagnostics(lags=15)

plt.show()

In [ ]:
# model_df = p+q+1 (drift)
# исключаем первый остаток .resid[1:] т.к. дифференцируем один раз
acorr_ljungbox(arima_drift_res.resid[1:], lags=[5], model_df=1+1+1)

In [ ]:
arima_drift_res.test_serial_correlation(method=None, lags=[5], df_adjust=True)

In [ ]:
lm_stat, lm_pval, f_stat, f_pval = het_arch(arima_drift_res.resid[1:], nlags=7, ddof=1+1+1)

lm_stat, lm_pval